In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
data = Path('./Resources/lending_data.csv')
df = pd.read_csv(data)


# Review the DataFrame
df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = df["loan_status"]

# Separate the X variable, the features
X = df.drop(columns="loan_status")

In [4]:
# Review the y variable Series
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [5]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Split the data into training and testing datasets by using `train_test_split`.

In [6]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(58152, 7)

In [7]:
# For training data
print("Training data distribution:")
print(y_train.value_counts())

# For test data
print("\nTest data distribution:")
print(y_test.value_counts())

Training data distribution:
loan_status
0    56277
1     1875
Name: count, dtype: int64

Test data distribution:
loan_status
0    18759
1      625
Name: count, dtype: int64


This shows:

There are 56,277 healthy loans (0's) in the training data.
There are 1,875 high-risk loans (1's) in the training data.


## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)


# Fit the model using training data
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [9]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
Results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
Results.head(10)

,Prediction,Actual
0,0,0
1,1,1
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


In [10]:
# Calculate accuracy score
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)

# Print the accuracy formatted to four decimal places
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.9939


### Step 3: Evaluate the model’s performance by doing the following:

* Generate a confusion matrix.

* Print the classification report.

In [11]:
# Create and save the confusion matrix for the testing data
confusion_matrix(y_test, predictions)

array([[18673,    86],
       [   32,   593]])

In [12]:
# Create and save the testing classification report
testing_report = classification_report(y_test, predictions)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      0.95      0.91       625

    accuracy                           0.99     19384
   macro avg       0.94      0.97      0.95     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The model excels at predicting healthy loans due to the large number of such instances (18,759), and it handles high-risk loans reasonably well, considering their small presence in the data (625). However, with a smaller class like high-risk loans, the precision of 0.87 suggests some high-risk loans are being incorrectly predicted as healthy loans, and there is slight room for improvement in precision for these high-risk cases.

Therefore, improving the precision of high-risk loan predictions (label 1) can be crucial for reducing false positives (i.e., predicting a loan as high-risk when it is not). We can acheive this through techniques like Handle Class Imbalance by Undersample the majority class (healthy loans) to reduce the class imbalance.


# Address the class imbalance in the dataset by applying RandomUnderSampler to reduce the overrepresentation of the majority class (healthy loans)

In [13]:
# installed imbalanced-learn package using pip install imbalanced-learn command line. 

from imblearn.under_sampling import RandomUnderSampler

#Apply undersampling only to the training data
undersampler = RandomUnderSampler(sampling_strategy='majority', random_state=1)
X_res, y_res = undersampler.fit_resample(X_train, y_train)



In [14]:
# Check distribution of the resampled training data
print("Resampled training data distribution:")
print(y_res.value_counts())


Resampled training data distribution:
loan_status
0    1875
1    1875
Name: count, dtype: int64


In [15]:
#Train a logistic regression model on the resampled (balanced) training dataset
model = LogisticRegression()
model.fit(X_res, y_res)

#Predict on the original test set (which is still imbalanced)
N_pred = model.predict(X_test)



In [16]:

# Display the accuracy score for the test dataset.
accuracy = accuracy_score(y_test, N_pred)

# Print the accuracy formatted to four decimal places
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9952


In [17]:
# Create and save the confusion matrix for the testing data
confusion_matrix(y_test, N_pred)

array([[18668,    91],
       [    2,   623]])

In [18]:
#Print the classification report to evaluate the model's performance
print(classification_report(y_test, N_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      1.00      0.93       625

    accuracy                           1.00     19384
   macro avg       0.94      1.00      0.96     19384
weighted avg       1.00      1.00      1.00     19384




Results: 
Recall for High-Risk Loans Improved After Resampling.  The recall increased from 0.95 to 1.00 after resampling, meaning that after resampling, the model is able to identify all high-risk loans (no false negatives).
Precision Remained the Same: The precision for high-risk loans did not change, meaning that the proportion of correctly predicted high-risk loans is consistent.
F1-Score Slightly Improved: The F1-score for high-risk loans increased from 0.91 to 0.93 after resampling, indicating an overall improvement in balancing precision and recall for this class.



---